In [ ]:
import panel as pn
pn.extension('vtk')

The ``VTK`` pane renders VTK objects and vtk.js files inside a panel, making it possible to interact with complex geometries in 3D.

#### Parameters:

For layout and styling related parameters see the [customization user guide](../../user_guide/Customization.ipynb).

* **``camera``** (dict): A dictionary reflecting the current state of the VTK camera
* **``enable_keybindings``** (bool): A boolean to activate/deactivate keybindings. Bound keys are:
  - s: set representation of all actors to *surface*
  - w: set representation of all actors to *wireframe*
  - v: set representation of all actors to *vertex*
  - r: center the actors and move the camera so that all actors are visible
  <br>**Warning**: These keybindings may not work as expected in a notebook context, if they interact with already bound keys
* **``object``** (str or object): Can be a string pointing to a local or remote file with a `.vtkjs` extension, or a `vtkRenderWindow` object  

___

The simplest way to construct a VTK pane is to give it a vtk.js file which it will serialize and embed in the plot. The ``VTK`` pane also supports the regular sizing options provided by Bokeh, including responsive sizing modes:

In [ ]:
dragon = pn.pane.VTK('https://raw.githubusercontent.com/Kitware/vtk-js/master/Data/StanfordDragon.vtkjs',
                     sizing_mode='stretch_width', height=400)
dragon

The ``VTK`` pane can also be updated like all other pane objects by replacing the ``object``:

In [ ]:
dragon.object = "https://github.com/Kitware/vtk-js-datasets/raw/master/data/vtkjs/TBarAssembly.vtkjs"

## Camera control

Once a VTK pane has been displayed it will automatically sync the camera state with the Pane object. We can read the camera state on the corresponding parameter:

```python
> dragon.camera

{'position': [-21.490090356222225, 14.44963146483641, 26.581314468858984],
 'focalPoint': [0, 4.969950199127197, 0],
 'viewUp': [0.17670012087160802, 0.9635684210080306, -0.20078088883170594],
 'directionOfProjection': [0.605834463228546,
  -0.2672449261957517,
  -0.749362897791989],
 'parallelProjection': False,
 'useHorizontalViewAngle': False,
 'viewAngle': 30,
 'parallelScale': 9.180799381276024,
 'clippingRange': [26.442079567041056, 44.714416678555395],
 'thickness': 1000,
 'windowCenter': [0, 0],
 'useOffAxisProjection': False,
 'screenBottomLeft': [-0.5, -0.5, -0.5],
 'screenBottomRight': [0.5, -0.5, -0.5],
 'screenTopRight': [0.5, 0.5, -0.5],
 'freezeFocalPoint': False,
 'useScissor': False,
 'projectionMatrix': None,
 'viewMatrix': None,
 'physicalTranslation': [0, -4.969950199127197, 0],
 'physicalScale': 9.180799381276024,
 'physicalViewUp': [0, 1, 0],
 'physicalViewNorth': [0, 0, -1],
 'mtime': 2237,
 'distance': 35.47188491341284}
```

This technique also makes it possible to link the camera of two or more VTK panes together:

In [ ]:
dragon1 = pn.pane.VTK('https://raw.githubusercontent.com/Kitware/vtk-js/master/Data/StanfordDragon.vtkjs',
                      height=400, sizing_mode='stretch_width')
dragon2 = pn.pane.VTK('https://raw.githubusercontent.com/Kitware/vtk-js/master/Data/StanfordDragon.vtkjs',
                      height=400, sizing_mode='stretch_width')

dragon1.jslink(dragon2, camera='camera')
dragon2.jslink(dragon1, camera='camera')

pn.Row(dragon1, dragon2)

and to modify the camera state in Python and trigger an update:

In [ ]:
if dragon.camera:
    dragon.camera['viewAngle'] = 50
    dragon.param.trigger('camera')

## Rendering VTK objects

In addition to support for plotting vtkjs files the VTK pane can also render objects defined using the ``vtk`` Python library.

There are slight differences with classical code generally used. As rendering of the objects and interactions with the view are handle by the VTK panel, we don't need to make a call to the `Render` method of the `vtkRenderWindow` (which would pop up the classical VTK window) or to specify a `vtkRenderWindowInteractor`. 

In [ ]:
import vtk
from vtk.util.colors import tomato

# This creates a polygonal cylinder model with eight circumferential
# facets.
cylinder = vtk.vtkCylinderSource()
cylinder.SetResolution(8)

# The mapper is responsible for pushing the geometry into the graphics
# library. It may also do color mapping, if scalars or other
# attributes are defined.
cylinderMapper = vtk.vtkPolyDataMapper()
cylinderMapper.SetInputConnection(cylinder.GetOutputPort())

# The actor is a grouping mechanism: besides the geometry (mapper), it
# also has a property, transformation matrix, and/or texture map.
# Here we set its color and rotate it -22.5 degrees.
cylinderActor = vtk.vtkActor()
cylinderActor.SetMapper(cylinderMapper)
cylinderActor.GetProperty().SetColor(tomato)
# We must set ScalarVisibilty to 0 to use tomato Color
cylinderMapper.SetScalarVisibility(0)
cylinderActor.RotateX(30.0)
cylinderActor.RotateY(-45.0)

# Create the graphics structure. The renderer renders into the render
# window.
ren = vtk.vtkRenderer()
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer(ren)

# Add the actors to the renderer, set the background and size
ren.AddActor(cylinderActor)
ren.SetBackground(0.1, 0.2, 0.4)

geom_pane = pn.pane.VTK(renWin, width=500, height=500)

geom_pane

We can also add additional actors to the plot and then trigger an update:

In [ ]:
sphere = vtk.vtkSphereSource()

sphereMapper = vtk.vtkPolyDataMapper()
sphereMapper.SetInputConnection(sphere.GetOutputPort())

sphereActor = vtk.vtkActor()
sphereActor.SetMapper(sphereMapper)
sphereActor.GetProperty().SetColor(tomato)
sphereMapper.SetScalarVisibility(0)
sphereActor.RotateX(30.0)
sphereActor.RotateY(-45.0)
sphereActor.SetPosition(0.5, 0.5, 0.5)

ren.AddActor(sphereActor)

geom_pane.param.trigger('object')